In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd
import numpy as np

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
def get_ride_id(x):
    year = x["tpep_pickup_datetime"].year
    month = x["tpep_pickup_datetime"].month
    index_val = str(x.name)
    ride_id = f'{year:04d}/{month:02d}_' + index_val
    return ride_id

df['ride_id'] = df.apply(lambda x: get_ride_id(x), axis = 1)
df

VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               1  2022-02-01 00:06:58   2022-02-01 00:19:24              1.0   
1               1  2022-02-01 00:38:22   2022-02-01 00:55:55              1.0   
2               1  2022-02-01 00:03:20   2022-02-01 00:26:59              1.0   
3               2  2022-02-01 00:08:00   2022-02-01 00:28:05              1.0   
4               2  2022-02-01 00:06:48   2022-02-01 00:33:07              1.0   
...           ...                  ...                   ...              ...   
2979426         2  2022-02-28 23:50:00   2022-03-01 00:06:00              NaN   
2979427         2  2022-02-28 23:06:57   2022-02-28 23:19:12              NaN   
2979428         2  2022-02-28 23:48:13   2022-03-01 00:03:33              NaN   
2979429         2  2022-02-28 23:56:41   2022-03-01 00:04:57              NaN   
2979430         2  2022-02-28 23:04:00   2022-02-28 23:10:00              NaN   

         trip_distance  RatecodeID store_and_fwd_flag PULocationID  \
0                 5.40         1.0                  N          138   
1                 6.40         1.0                  N          138   
2                12.50         1.0                  N          138   
3                 9.88         1.0                  N          239   
4                12.16         1.0                  N          138   
...                ...         ...                ...          ...   
2979426           3.40         NaN               None          163   
2979427           3.48         NaN               None          141   
2979428           3.05         NaN               None          161   
2979429           2.62         NaN               None          141   
2979430           1.41         NaN               None          237   

        DOLocationID  payment_type  ...  extra  mta_tax  tip_amount  \
0                252             1  ...   1.75      0.5        3.90   
1                 41             2  ...   1.75      0.5        0.00   
2                200             2  ...   1.75      0.5        0.00   
3                200             2  ...   0.50      0.5        0.00   
4                125             1  ...   0.50      0.5        8.11   
...              ...           ...  ...    ...      ...         ...   
2979426          193             0  ...   0.00      0.5        0.00   
2979427            4             0  ...   0.00      0.5        2.00   
2979428          151             0  ...   0.00      0.5        3.81   
2979429          226             0  ...   0.00      0.5        1.71   
2979430          142             0  ...   0.00      0.5        2.57   

         tolls_amount  improvement_surcharge  total_amount  \
0                0.00                    0.3         23.45   
1                6.55                    0.3         30.10   
2                6.55                    0.3         44.60   
3                3.00                    0.3         34.80   
4                0.00                    0.3         48.66   
...               ...                    ...           ...   
2979426          0.00                    0.3         17.67   
2979427          0.00                    0.3         19.81   
2979428          0.00                    0.3         21.49   
2979429          0.00                    0.3         17.54   
2979430          0.00                    0.3         14.67   

         congestion_surcharge  airport_fee   duration          ride_id  
0                         0.0         1.25  12.433333        2022/02_0  
1                         0.0         1.25  17.550000        2022/02_1  
2                         0.0         1.25  23.650000        2022/02_2  
3                         2.5         0.00  20.083333        2022/02_3  
4                         2.5         1.25  26.316667        2022/02_4  
...                       ...          ...        ...              ...  
2979426                   NaN          NaN  16.000000  2022/02_2979426  
2979427      

In [9]:
df_result = df[["ride_id"]]
df_result["predictions"] = y_pred
df_result

/tmp/ipykernel_90388/1303464442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result["predictions"] = y_pred


ride_id  predictions
0              2022/02_0    18.527783
1              2022/02_1    23.065782
2              2022/02_2    33.686359
3              2022/02_3    23.757436
4              2022/02_4    21.492904
...                  ...          ...
2979426  2022/02_2979426    12.038225
2979427  2022/02_2979427    11.441569
2979428  2022/02_2979428    11.890459
2979429  2022/02_2979429    15.102681
2979430  2022/02_2979430     9.460592

[2918187 rows x 2 columns]

In [10]:
output_file = "./result.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)